# Adam Forestier
## Last Updated: September 20, 2023

This File covers the following:
- Google's beginner walkthrough to TensorFlow 2 from: https://www.tensorflow.org/tutorials/quickstart/beginner.
- All credit to Google, I am just typing out the walkthrough and saving notes from their documentation

In [45]:
import tensorflow as tf

### Load Dataset

In [46]:
data = tf.keras.datasets.mnist

### Scale Data
Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers

In [47]:
(x_train, y_train), (x_test, y_test) = data.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

### Build ML Model

In [48]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(units=128, activation='relu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(units=10)
])

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [49]:
y_pred = model(x_train[:1]).numpy()
y_pred

array([[ 0.81675273,  0.9600187 , -0.07746734, -0.5565133 , -0.9071664 ,
        -0.44079536, -0.5822122 ,  0.01224104, -0.5827246 ,  0.77542925]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [50]:
tf.nn.softmax(logits=y_pred).numpy()

array([[0.193074  , 0.22281449, 0.07895306, 0.04890148, 0.03443779,
        0.05490066, 0.04766077, 0.08636322, 0.04763636, 0.18525812]],
      dtype=float32)

Define a loss function for training using losses.SparseCategoricalCrossentropy:

In [51]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [52]:
loss_function(y_true=y_train[:1], y_pred=y_pred).numpy()

2.9022298

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy

In [53]:
model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

### Train and Evaluate Model

Use Model.fit method to adjust your model parameters and minimize the loss

In [54]:
model.fit(x=x_train, y=y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.3015 - accuracy: 0.9115
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1457 - accuracy: 0.9566
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1096 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0877 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0733 - accuracy: 0.9774


The Model.evaluate method checks the model's performance, usually on a validation set or test set.

In [55]:
model.evaluate(x=x_test, y=y_test, verbose=2)

313/313 - 1s - loss: 0.0740 - accuracy: 0.9776 - 737ms/epoch - 2ms/step


[0.07401582598686218, 0.9775999784469604]

The image classifier is now trained to ~98% accuracy on this dataset

If you want your model to return a probabilyt, you can wrap the trained model and attach the softmax to it

In [56]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [57]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.76354272e-08, 5.39315981e-10, 1.60939635e-06, 2.08792218e-04,
        1.65343832e-11, 9.13349368e-07, 1.11296938e-13, 9.99784172e-01,
        3.29000585e-07, 4.05780020e-06],
       [2.04308481e-08, 1.13999944e-04, 9.99804199e-01, 4.92126674e-05,
        5.97379727e-15, 3.23519889e-05, 1.58236475e-07, 3.24885890e-14,
        1.30341277e-07, 6.20811750e-15],
       [8.42714769e-08, 9.99146104e-01, 3.00845655e-04, 6.82761765e-06,
        2.07045923e-05, 2.00284126e-06, 7.69315466e-06, 3.80568876e-04,
        1.33563633e-04, 1.60510240e-06],
       [9.99847651e-01, 2.34843828e-11, 4.44821890e-05, 4.82203646e-08,
        2.88531965e-07, 1.95465236e-05, 6.34968019e-05, 2.32823295e-05,
        1.79512725e-07, 9.86362920e-07],
       [4.14379713e-07, 2.79512169e-09, 3.49753122e-06, 1.66716940e-09,
        9.98272061e-01, 3.06715748e-07, 2.48826012e-07, 2.77052721e-04,
        1.21916671e-07, 1.44639926e-03]], dtype=float32)>